In [1]:
import os
os.chdir('../../src/')
from visualization.SIR_Plot import make_SIR_graph, make_beds_graph

In [2]:
os.chdir('models')
import simulate_pandemic as simp
from actions import costs, city_restrictions
from MCFS import mcts, treeNode
from CMDP import CovidState

Loading Graph...Done!


In [3]:
from tqdm import tqdm
import numpy as np
import pickle as pkl
import datetime

In [4]:
import plotly.graph_objects as go
import pandas as pd

def make_beds_graph(data, actions, step_size, title, max_range=None, color_map=None, make_df=True):
    fig = go.Figure()
    
    if make_df:
        data = pd.DataFrame([pd.Series(d).value_counts() for d in data] + [pd.Series(data[-1]).value_counts()])
        data.fillna(0, inplace=True)
        data

    color_map = {
        'Lockdown':          'rgb(0.83, 0.13, 0.15)',
        'Hard Quarantine':    'rgb(0.85, 0.35, 0.13)',
        'Light Quarantine':   'rgb(0.97, 0.91, 0.56)',
        'Social Distancing':  'rgb(0.67, 0.88, 0.69)',
        'Unrestricted':        'rgb(0.86, 0.86, 0.86)'    
    }

    pop = 55492
    
    actions = list(map(color_map.get,  actions))
    
    
    x = list(range(len(data) + 1))
    
    #\definecolor{royalblue(web)}{rgb}{0.25, 0.41, 0.88}
    fig.add_trace(go.Scatter(x=x, y=data[3]/pop, name='hospitalized', line_color = 'rgb(0.25, 0.41, 0.88)',
                            line=dict(width=3.5),  
                            ))
    
    fig.add_trace(go.Scatter(x=x, y=(len(data))*[0.0015], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2),
                            ))
    # ACTIONS BACKGROUND################################################################################
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=(step_size*i),
                y0=0,
                x1=step_size*(i+1),
                y1=1,
                fillcolor=a,
                opacity=0.45,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)]
    )

    fig.update_layout(xaxis={'showgrid': False,},
                      yaxis = {'showgrid': False, 'zeroline': False,},
                      showlegend=True, hovermode="x")
    
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = list(range(0, len(data)+1, 14)),
            ticktext = list(range(0, int(len(data)/7) + 1, 2)),
            title = 'Weeks',
            )
        )
    fig.update_layout(showlegend=False,
        plot_bgcolor='rgb(255,255,255)',
        margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=0
    ))
    
    fig.update_yaxes(automargin=True)
    fig.write_image(f"{title}.pdf")

    fig.show()
    
    return data

In [5]:
def run_full_mcst(rolloutPolicy='rolloutPolicy', horizon=1, bruteForce=False,
                  sims_per_leaf=10, n_jobs = 6, step_size = 7, days = 210):

    pop_matrix = simp.init_infection()
    data = []
    actions = []

    for day in tqdm(range(1, days+1)):
        #if less than 20% still susceptible, break simulation
        if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break            
            
        
        # Choose a new policy at each week
        if day % step_size == 1:                    
            tree = mcts(sims_per_leaf=sims_per_leaf, step_size=step_size, horizon=horizon,
                        n_jobs=n_jobs, rolloutPolicy=rolloutPolicy, pop_matrix=pop_matrix, bruteForce=bruteForce)

            root = treeNode(CovidState(actions=[], day=day), parent=None)
            action, best_node = tree.search(root)

            actions.append(action)
            restrictions = city_restrictions[action]

        pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
        pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix,key=lambda x: x[0]))[:,1]) 
    
    return data, actions, tree

In [ ]:
horizon = 4
sims = 6
days = 364
bf = False

data, actions, tree = run_full_mcst(horizon=horizon, sims_per_leaf=sims, days=days, step_size=7, n_jobs=6, bruteForce=bf)

date = datetime.datetime.now()
date_str = f'{date.month}_{date.day}_{date.hour}_{date.minute}'

df = make_beds_graph(data, actions, 7, f'../../data/MCTS_Results/plots/hosp_cost_H{horizon}_N{sims}_D{days}_bf{bf}_{date_str}')

with open(f'../../data/MCTS_Results/pickles/looser_cost_H{horizon}_N{sims}_D{days}_bf{bf}_{date_str}', 'wb') as f:
    pkl.dump((data, actions, tree), f)

  2%|█▎                                                                              | 6/364 [01:22<3:59:40, 40.17s/it]